In [35]:
import pandas as pd
#tips,类似账户寻找

In [36]:
operation_train=pd.read_csv('data/operation_train_new.csv')
transaction_train=pd.read_csv('data/transaction_train_new.csv')
tag_train=pd.read_csv('data/tag_train_new.csv')

operation_test=pd.read_csv('data/test_transaction_round2.csv')
transaction_test=pd.read_csv('data/test_operation_round2.csv')
operation_data=operation_train.append(operation_test).reset_index(drop=True)
transaction_data=transaction_train.append(transaction_test).reset_index(drop=True)

C:\Users\yzzh1\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\yzzh1\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [37]:
#删除一些特征
del operation_train['version']
del operation_train['os']
del operation_train['device1']
del operation_train['success']
del operation_train['time']  #暂时先不考虑具体时间点
del operation_train['mac1']
del operation_train['ip2']
del operation_train['wifi']
del operation_train['ip2_sub']

del transaction_train['time']
del transaction_train['trans_amt']
del transaction_train['code1']
del transaction_train['code2']
del transaction_train['acc_id1']
del transaction_train['device1']
del transaction_train['acc_id2']
del transaction_train['acc_id3']
del transaction_train['market_code']   
del transaction_train['market_type']
#添加一些特征
#测试集合中device_code出现10个不同以上或者mac2 20个以上，'device_code3'出现20个以上的一律预测为黑产
#商户id在其中，必定为黑户['1f72814f76a984fa','0e90f47392008def','5776870b5747e14e','6d55ccc689b910ee','8b3f74a1391b5427','0bdfe811a0e60e0f','e1bad078cf58a817','ad0aa55c2ea8ff76','2260d61b622795fb']

def str_mapper(str0):
    if str0[:6]=='IPHONE':
        return str0[:8]
    elif str0[:4]=='VIVO':
        return 'VIVO'
    elif str0[:4]=='OPPO':
        return 'OPPO'
    elif str0[:6]=='HUAWEI':
        return 'HUAWEI'
    elif str0[:2]=='MI':
        return 'MI'
    else:
        return 'OTHER'
    
operation_train['device2']=operation_train['device2'].astype(str)
operation_train['device2']=operation_train['device2'].map(str_mapper)
transaction_train['device2']=transaction_train['device2'].astype(str)
transaction_train['device2']=transaction_train['device2'].map(str_mapper)

In [98]:
transaction_train_bad.groupby('amt_src2').size().reset_index().sort_values(0,ascending=False)
transaction_train_bad.groupby('UID')['trans_type2'].nunique().reset_index().sort_values('trans_type2',ascending=False).head(3500)

,amt_src2,0
24,9fefed0a981dcb7a,14178
25,a2aa73cdb6621133,4048
27,c4ec9622cf5c6e55,2993
29,cf6e3a074407c379,2734
23,9a8ee16bde15e38a,1723
36,e98d66cbb6c099b5,515
6,1ca672c7cb34af43,259
4,12e06cb81b0fcf2a,196
35,e86309711bd84312,68
16,7bbedaeb92ef822e,65


In [58]:
####
tag_bad=tag_train[tag_train['Tag']==1]
tag_good=tag_train[tag_train['Tag']!=1]

####
operation_train_bad=pd.merge(operation_train,tag_bad,on='UID',how='inner')
operation_train_good=pd.merge(operation_train,tag_good,on='UID',how='inner')
transaction_train_bad=pd.merge(transaction_train,tag_bad,on='UID',how='inner')
transaction_train_good=pd.merge(transaction_train,tag_good,on='UID',how='inner')
#transaction_train_good.groupby('UID')['ip1_sub'].nunique().reset_index().sort_values('ip1_sub',ascending=False).tail(500)

In [38]:
#训练集数量频词特征
operation_feature_list=['device2','mode','device_code1','device_code3','mac2','ip1','geo_code','ip1_sub']
device_list=['IPHONE 5','IPHONE 6','VIVO','OPPO','HUAWEI','MI','OTHER']
mode_list=['c8741ce15ceac2a4','acfaded7e04e7ba0','79f8b86938180c3c','b501fa4fc58206b9','3b02bcd4b685d7a8','2f3e878175e34d9c','1c341176507fbd9b','00094ae2a1d62504','20a91b45ef8f8221']
operation_data_feature=operation_data.groupby('UID').size().reset_index()
del operation_data_feature[0]
operation_data_feature['operation_total_num']=operation_data.groupby('UID').size().reset_index()[0]
operation_data_feature['operation_day_num']=operation_data.groupby('UID')['day'].nunique().reset_index()['day']
operation_data_feature['operation_device2_num']=operation_data.groupby('UID')['device2'].nunique().reset_index()['device2']
operation_data_feature['operation_mode_num']=operation_data.groupby('UID')['mode'].nunique().reset_index()['mode']
operation_data_feature['operation_device_code1_num']=operation_data.groupby('UID')['device_code1'].nunique().reset_index()['device_code1']
operation_data_feature['operation_device_code3_num']=operation_data.groupby('UID')['device_code3'].nunique().reset_index()['device_code3']
operation_data_feature['operation_mac2_num']=operation_data.groupby('UID')['mac2'].nunique().reset_index()['mac2']
operation_data_feature['operation_ip1_num']=operation_data.groupby('UID')['ip1'].nunique().reset_index()['ip1']
operation_data_feature['operation_geo_code_num']=operation_data.groupby('UID')['geo_code'].nunique().reset_index()['geo_code']
operation_data_feature['operation_ip1_sub_num']=operation_data.groupby('UID')['ip1_sub'].nunique().reset_index()['ip1_sub']
#operation_train_feature['operation_device_code2_num']=operation_train.groupby('UID')['device_code2'].nunique().reset_index()['device_code2']
#是否缺失
operation_data_feature['operation_device_code1_isnull']=operation_data_feature['operation_device_code1_num'].apply(lambda x:0 if x==0 else 1)
operation_data_feature['operation_device_code3_isnull']=operation_data_feature['operation_device_code3_num'].apply(lambda x:0 if x==0 else 1)
operation_data_feature['operation_mac2_isnull']=operation_data_feature['operation_mac2_num'].apply(lambda x:0 if x==0 else 1)
operation_data_feature['operation_geo_code_isnull']=operation_data_feature['operation_geo_code_num'].apply(lambda x:0 if x==0 else 1)

for i in range(9):
    df=operation_data[operation_data['mode']==mode_list[i]].groupby('UID').size().reset_index()
    df['operation_mode_'+str(i)]=df[0]
    del df[0]
    operation_data_feature=pd.merge(operation_data_feature,df,on='UID',how='left')
for i in range(7):
    df=operation_data[operation_data['device2']==device_list[i]].groupby('UID').size().reset_index()
    df['operation_device2_'+str(i)]=df[0]
    del df[0]
    operation_data_feature=pd.merge(operation_data_feature,df,on='UID',how='left')
    
operation_data_feature.fillna(0)



for feature1 in operation_feature_list:
    for feature2 in operation_feature_list:
        if feature1!=feature2:
            operation_data_feature['transaction_'+feature1+'_'+feature2+'_num']=operation_data[['UID',feature1]].merge(operation_data.groupby([feature1])[feature2].nunique().reset_index(),on=feature1,how='left')[['UID',feature2]].groupby('UID')[feature2].sum().reset_index()[feature2] 

In [39]:
device_list=['IPHONE 5','IPHONE 6','VIVO','OPPO','HUAWEI','MI','OTHER']
transaction_feature_list=['channel','merchant','device_code1','device_code3','ip1','geo_code','ip1_sub']
transaction_data_feature=transaction_data.groupby('UID').size().reset_index()
del transaction_data_feature[0]
transaction_data_feature['transaction_total_num']=transaction_data.groupby('UID').size().reset_index()[0]
transaction_data_feature['operation_device2_num']=transaction_data.groupby('UID')['device2'].nunique().reset_index()['device2']
transaction_data_feature['transaction_day_num']=transaction_data.groupby('UID')['day'].nunique().reset_index()['day']
transaction_data_feature['transaction_channel_num']=transaction_data.groupby('UID')['channel'].nunique().reset_index()['channel']
transaction_data_feature['transaction_merchant_num']=transaction_data.groupby('UID')['merchant'].nunique().reset_index()['merchant']
transaction_data_feature['transaction_device_code1_num']=transaction_data.groupby('UID')['device_code1'].nunique().reset_index()['device_code1']
transaction_data_feature['transaction_device_code3_num']=transaction_data.groupby('UID')['device_code3'].nunique().reset_index()['device_code3']
transaction_data_feature['transaction_ip1_num']=transaction_data.groupby('UID')['ip1'].nunique().reset_index()['ip1']

transaction_data_feature['transaction_trans_amt_max']=transaction_data.groupby('UID')['trans_amt'].max().reset_index()['trans_amt']
transaction_data_feature['transaction_trans_amt_min']=transaction_data.groupby('UID')['trans_amt'].min().reset_index()['trans_amt']
transaction_data_feature['transaction_trans_amt_sum']=transaction_data.groupby('UID')['trans_amt'].sum().reset_index()['trans_amt']
transaction_data_feature['transaction_trans_amt_mean']=transaction_data.groupby('UID')['trans_amt'].mean().reset_index()['trans_amt']
transaction_data_feature['transaction_trans_amt_std']=transaction_data.groupby('UID')['trans_amt'].mean().reset_index()['trans_amt']

transaction_data_feature['transaction_trans_bal_max']=transaction_data.groupby('UID')['bal'].max().reset_index()['bal']
transaction_data_feature['transaction_trans_bal_min']=transaction_data.groupby('UID')['bal'].min().reset_index()['bal']
transaction_data_feature['transaction_trans_bal_sum']=transaction_data.groupby('UID')['bal'].sum().reset_index()['bal']
transaction_data_feature['transaction_trans_bal_mean']=transaction_data.groupby('UID')['bal'].mean().reset_index()['bal']
transaction_data_feature['transaction_trans_amt_std']=transaction_data.groupby('UID')['bal'].mean().reset_index()['bal']




transaction_data_feature['transaction_geo_code_num']=transaction_data.groupby('UID')['geo_code'].nunique().reset_index()['geo_code']
transaction_data_feature['transaction_ip1_sub_num']=transaction_data.groupby('UID')['ip1_sub'].nunique().reset_index()['ip1_sub']
channel_list=[102,140,118,106,119]
amt_list1=['155c9e1c32bd0fa2','c5fc631370cabc0d','fd4d2d1006a95637','4d7831c6f695ab19','8c9987909b3e95a4','f29829bc82459191','a571c7fda8b7df37','27c42480134c0d02']
trans_type1_list=['c2f2023d279665b2','6d55c54c8b1056fb','61bfb66c928f36ac','ced62357ad496957','26bcf43a19df14c8']
trans_type2_list=[102.0,103.0,104.0,105.0]
amt_list2=['9fefed0a981dcb7a','a2aa73cdb6621133','c4ec9622cf5c6e55','cf6e3a074407c379','9a8ee16bde15e38a','e98d66cbb6c099b5','1ca672c7cb34af43','12e06cb81b0fcf2a','e2a24888724d3e89','e7bed445b4f874ac','e98d66cbb6c099b5','12e06cb81b0fcf2a','4a4fdbd7ae5655cd','8d2ff85b6fd5dc78','d46a2a9577fb52c3','fbf6bf3c8927414c','7bbedaeb92ef822e']
for i in range(5):
    df=transaction_data[transaction_data['channel']==channel_list[i]].groupby('UID').size().reset_index()
    df['transaction_channel_'+str(i)]=df[0]
    del df[0]
    transaction_data_feature=pd.merge(transaction_data_feature,df,on='UID',how='left')
    
for i in range(8):
    df=transaction_data[transaction_data['amt_src1']==amt_list1[i]].groupby('UID').size().reset_index()
    df['transaction_amt_src1_'+str(i)]=df[0]
    del df[0]
    transaction_data_feature=pd.merge(transaction_data_feature,df,on='UID',how='left')
for i in range(17):
    df=transaction_data[transaction_data['amt_src2']==amt_list2[i]].groupby('UID').size().reset_index()
    df['transaction_amt_src2_'+str(i)]=df[0]
    del df[0]
    transaction_data_feature=pd.merge(transaction_data_feature,df,on='UID',how='left')
    
for i in range(5):
    df=transaction_data[transaction_data['trans_type1']==trans_type1_list[i]].groupby('UID').size().reset_index()
    df['transaction_trans_type1_'+str(i)]=df[0]
    del df[0]
    transaction_data_feature=pd.merge(transaction_data_feature,df,on='UID',how='left')

for i in range(4):
    df=transaction_data[transaction_data['trans_type2']==trans_type2_list[i]].groupby('UID').size().reset_index()
    df['transaction_trans_type2_'+str(i)]=df[0]
    del df[0]
    transaction_data_feature=pd.merge(transaction_data_feature,df,on='UID',how='left')
for i in range(7):
    df=transaction_data[transaction_data['device2']==device_list[i]].groupby('UID').size().reset_index()
    df['transaction_device2_'+str(i)]=df[0]
    del df[0]

transaction_data_feature['transaction_device_code1_isnull']=transaction_data_feature['transaction_device_code1_num'].apply(lambda x:0 if x==0 else 1)
transaction_data_feature['transaction_device_code3_isnull']=transaction_data_feature['transaction_device_code3_num'].apply(lambda x:0 if x==0 else 1)
transaction_data_feature['transaction_geo_code_isnull']=transaction_data_feature['transaction_geo_code_num'].apply(lambda x:0 if x==0 else 1)
transaction_data_feature.fillna(0)


for feature1 in transaction_feature_list:
    for feature2 in transaction_feature_list:
        if feature1!=feature2:
            transaction_data_feature['transaction_'+feature1+'_'+feature2+'_num']=transaction_data[['UID',feature1]].merge(transaction_data.groupby([feature1])[feature2].nunique().reset_index(),on=feature1,how='left')[['UID',feature2]].groupby('UID')[feature2].sum().reset_index()[feature2] 

In [40]:
transaction_test_feature=transaction_data_feature[~transaction_data_feature['UID'].isin(tag_train['UID'])]
transaction_train_feature=transaction_data_feature[transaction_data_feature['UID'].isin(tag_train['UID'])]

operation_test_feature=operation_data_feature[~operation_data_feature['UID'].isin(tag_train['UID'])]
operation_train_feature=operation_data_feature[operation_data_feature['UID'].isin(tag_train['UID'])]

In [41]:
train_feature=pd.merge(operation_train_feature,transaction_train_feature,how='outer',on='UID')
train_feature.fillna(0)

,UID,operation_total_num,operation_day_num,operation_device2_num_x,operation_mode_num,operation_device_code1_num,operation_device_code3_num,operation_mac2_num,operation_ip1_num,operation_geo_code_num,...,transaction_geo_code_device_code1_num_y,transaction_geo_code_device_code3_num_y,transaction_geo_code_ip1_num_y,transaction_geo_code_ip1_sub_num_y,transaction_ip1_sub_channel_num,transaction_ip1_sub_merchant_num,transaction_ip1_sub_device_code1_num_y,transaction_ip1_sub_device_code3_num_y,transaction_ip1_sub_ip1_num_y,transaction_ip1_sub_geo_code_num_y
0,10000,9.0,2.0,1.0,3.0,0.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,4.0,2.0,0.0
1,10001,67.0,8.0,1.0,6.0,1.0,1.0,1.0,9.0,3.0,...,968.0,266.0,3838.0,2476.0,43.0,167.0,294.0,90.0,362.0,262.0
2,10002,11.0,1.0,1.0,7.0,1.0,0.0,2.0,1.0,1.0,...,1041.0,60.0,2154.0,504.0,6.0,75.0,102.0,15.0,108.0,48.0
3,10003,15.0,1.0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,...,2.0,0.0,2.0,2.0,5.0,5.0,5.0,0.0,5.0,3.0
4,10004,34.0,6.0,1.0,9.0,1.0,0.0,1.0,6.0,1.0,...,306.0,16.0,454.0,360.0,13.0,19.0,26.0,7.0,35.0,26.0
5,10006,7.0,2.0,1.0,3.0,1.0,0.0,1.0,2.0,1.0,...,140.0,14.0,319.0,278.0,11.0,20.0,38.0,17.0,50.0,50.0
6,10007,13.0,2.0,1.0,3.0,1.0,0.0,1.0,3.0,1.0,...,4.0,8.0,36.0,36.0,6.0,106.0,130.0,98.0,172.0,78.0
7,10008,30.0,2.0,1.0,8.0,1.0,0.0,1.0,3.0,1.0,...,507.0,54.0,1911.0,1659.0,40.0,148.0,199.0,31.0,221.0,163.0
8,10009,12.0,3.0,1.0,2.0,1.0,0.0,1.0,3.0,1.0,...,54.0,24.0,221.0,176.0,13.0,161.0,282.0,144.0,409.0,209.0
9,10010,8.0,1.0,1.0,3.0,1.0,0.0,1.0,1.0,1.0,...,52.0,4.0,116.0,92.0,8.0,12.0,16.0,8.0,24.0,16.0


In [42]:
test_feature=pd.merge(operation_test_feature,transaction_test_feature,how='outer',on='UID')
test_feature.fillna(0)

,UID,operation_total_num,operation_day_num,operation_device2_num_x,operation_mode_num,operation_device_code1_num,operation_device_code3_num,operation_mac2_num,operation_ip1_num,operation_geo_code_num,...,transaction_geo_code_device_code1_num_y,transaction_geo_code_device_code3_num_y,transaction_geo_code_ip1_num_y,transaction_geo_code_ip1_sub_num_y,transaction_ip1_sub_channel_num,transaction_ip1_sub_merchant_num,transaction_ip1_sub_device_code1_num_y,transaction_ip1_sub_device_code3_num_y,transaction_ip1_sub_ip1_num_y,transaction_ip1_sub_geo_code_num_y
0,100000,6.0,2.0,1.0,0.0,1.0,0.0,0.0,2.0,1.0,...,660.0,99.0,1056.0,924.0,0.0,0.0,218.0,24.0,33.0,33.0
1,100001,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,504.0,104.0,1016.0,768.0,0.0,0.0,264.0,48.0,300.0,192.0
2,100002,18.0,5.0,4.0,0.0,4.0,0.0,0.0,9.0,3.0,...,2889.0,1758.0,10209.0,8062.0,87.0,422.0,540.0,806.0,1111.0,595.0
3,100003,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,574.0,322.0,2303.0,1750.0,14.0,105.0,203.0,98.0,287.0,231.0
4,100004,2.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,32.0,12.0,76.0,72.0,0.0,0.0,4.0,4.0,4.0,4.0
5,100005,6.0,4.0,1.0,0.0,1.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,132.0,394.0,775.0,646.0,1124.0,806.0
6,100006,2.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,144.0,18.0,585.0,288.0,22.0,123.0,182.0,25.0,197.0,102.0
7,100007,4.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,644.0,154.0,1064.0,798.0,40.0,940.0,7180.0,2120.0,520.0,2320.0
8,100008,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,186.0,39.0,459.0,159.0,12.0,36.0,150.0,30.0,234.0,78.0
9,100009,9.0,3.0,3.0,0.0,2.0,1.0,0.0,5.0,1.0,...,165.0,13.0,221.0,127.0,15.0,105.0,75.0,66.0,189.0,68.0


In [43]:
train_feature=train_feature.sort_values('UID').reset_index(drop=True)
test_feature=test_feature.sort_values('UID').reset_index(drop=True)

In [44]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import numpy as np
param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 11,
         "metric": 'binary_logloss',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 4,
         "random_state": 4590}
folds = StratifiedKFold(n_splits=20, shuffle=True, random_state=4590)
oof = np.zeros(len(train_feature))
predictions = np.zeros(len(test_feature))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_feature,tag_train['Tag'])):
    print("fold {}".format(fold_))
    trn_data = lgb.Dataset(train_feature.iloc[trn_idx], label=tag_train['Tag'].iloc[trn_idx])
    val_data = lgb.Dataset(train_feature.iloc[val_idx], label=tag_train['Tag'].iloc[val_idx])

    num_round = 60000
    clf = lgb.train(param, trn_data, num_round, valid_sets = val_data, verbose_eval=100, early_stopping_rounds = 2000)
    oof[val_idx] = clf.predict(train_feature.iloc[val_idx], num_iteration=clf.best_iteration)
    predictions += clf.predict(test_feature, num_iteration=clf.best_iteration) / folds.n_splits

fold 0
Training until validation scores don't improve for 2000 rounds.
[100]	valid_0's binary_logloss: 0.181242
[200]	valid_0's binary_logloss: 0.132175
[300]	valid_0's binary_logloss: 0.113719
[400]	valid_0's binary_logloss: 0.105623
[500]	valid_0's binary_logloss: 0.101819
[600]	valid_0's binary_logloss: 0.0993964
[700]	valid_0's binary_logloss: 0.0978989
[800]	valid_0's binary_logloss: 0.0967259
[900]	valid_0's binary_logloss: 0.0963674
[1000]	valid_0's binary_logloss: 0.0959901
[1100]	valid_0's binary_logloss: 0.0959312
[1200]	valid_0's binary_logloss: 0.0956771
[1300]	valid_0's binary_logloss: 0.095409
[1400]	valid_0's binary_logloss: 0.095407
[1500]	valid_0's binary_logloss: 0.0953701
[1600]	valid_0's binary_logloss: 0.0952956
[1700]	valid_0's binary_logloss: 0.0951513
[1800]	valid_0's binary_logloss: 0.0955004
[1900]	valid_0's binary_logloss: 0.0956431
[2000]	valid_0's binary_logloss: 0.095915
[2100]	valid_0's binary_logloss: 0.0960179
[2200]	valid_0's binary_logloss: 0.096487
[

[100]	valid_0's binary_logloss: 0.177624
[200]	valid_0's binary_logloss: 0.129614
[300]	valid_0's binary_logloss: 0.111542
[400]	valid_0's binary_logloss: 0.103175
[500]	valid_0's binary_logloss: 0.0988102
[600]	valid_0's binary_logloss: 0.0966025
[700]	valid_0's binary_logloss: 0.0954143
[800]	valid_0's binary_logloss: 0.0944828
[900]	valid_0's binary_logloss: 0.0935132
[1000]	valid_0's binary_logloss: 0.0930017
[1100]	valid_0's binary_logloss: 0.0927258
[1200]	valid_0's binary_logloss: 0.0925546
[1300]	valid_0's binary_logloss: 0.0923218
[1400]	valid_0's binary_logloss: 0.0923402
[1500]	valid_0's binary_logloss: 0.0924297
[1600]	valid_0's binary_logloss: 0.0923991
[1700]	valid_0's binary_logloss: 0.092648
[1800]	valid_0's binary_logloss: 0.0929938
[1900]	valid_0's binary_logloss: 0.0932342
[2000]	valid_0's binary_logloss: 0.0933
[2100]	valid_0's binary_logloss: 0.0936465
[2200]	valid_0's binary_logloss: 0.0940127
[2300]	valid_0's binary_logloss: 0.0945609
[2400]	valid_0's binary_logl

[1100]	valid_0's binary_logloss: 0.110297
[1200]	valid_0's binary_logloss: 0.110088
[1300]	valid_0's binary_logloss: 0.109892
[1400]	valid_0's binary_logloss: 0.109985
[1500]	valid_0's binary_logloss: 0.110407
[1600]	valid_0's binary_logloss: 0.110874
[1700]	valid_0's binary_logloss: 0.111252
[1800]	valid_0's binary_logloss: 0.111529
[1900]	valid_0's binary_logloss: 0.11175
[2000]	valid_0's binary_logloss: 0.112276
[2100]	valid_0's binary_logloss: 0.112885
[2200]	valid_0's binary_logloss: 0.113573
[2300]	valid_0's binary_logloss: 0.114192
[2400]	valid_0's binary_logloss: 0.114968
[2500]	valid_0's binary_logloss: 0.115466
[2600]	valid_0's binary_logloss: 0.116457
[2700]	valid_0's binary_logloss: 0.117451
[2800]	valid_0's binary_logloss: 0.118146
[2900]	valid_0's binary_logloss: 0.118872
[3000]	valid_0's binary_logloss: 0.119828
[3100]	valid_0's binary_logloss: 0.12081
[3200]	valid_0's binary_logloss: 0.12163
Early stopping, best iteration is:
[1244]	valid_0's binary_logloss: 0.109814
fo

[3400]	valid_0's binary_logloss: 0.100086
[3500]	valid_0's binary_logloss: 0.100822
Early stopping, best iteration is:
[1599]	valid_0's binary_logloss: 0.0923666
fold 16
Training until validation scores don't improve for 2000 rounds.
[100]	valid_0's binary_logloss: 0.177334
[200]	valid_0's binary_logloss: 0.128825
[300]	valid_0's binary_logloss: 0.109791
[400]	valid_0's binary_logloss: 0.101274
[500]	valid_0's binary_logloss: 0.097037
[600]	valid_0's binary_logloss: 0.0945446
[700]	valid_0's binary_logloss: 0.0929921
[800]	valid_0's binary_logloss: 0.0914858
[900]	valid_0's binary_logloss: 0.0902968
[1000]	valid_0's binary_logloss: 0.0892991
[1100]	valid_0's binary_logloss: 0.0885422
[1200]	valid_0's binary_logloss: 0.0882104
[1300]	valid_0's binary_logloss: 0.087644
[1400]	valid_0's binary_logloss: 0.0871446
[1500]	valid_0's binary_logloss: 0.0870603
[1600]	valid_0's binary_logloss: 0.0867078
[1700]	valid_0's binary_logloss: 0.0863946
[1800]	valid_0's binary_logloss: 0.0861441
[1900]	

In [45]:
test_feature=test_feature[['UID']]
test_feature['Tag']=predictions
res=test_feature[['UID','Tag']]

In [47]:
res.to_csv('data/result.csv')